In [1]:
!wget http://d28rh4a8wq0iu5.cloudfront.net/ads1/data/chr1.GRCh38.excerpt.fasta

--2025-02-10 15:49:10--  http://d28rh4a8wq0iu5.cloudfront.net/ads1/data/chr1.GRCh38.excerpt.fasta
Resolving d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)... 13.227.44.33, 13.227.44.207, 13.227.44.91, ...
Connecting to d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)|13.227.44.33|:80... connected.
200 OKequest sent, awaiting response... 
Length: 810105 (791K) [application/octet-stream]
Saving to: ‘chr1.GRCh38.excerpt.fasta’

chr1.GRCh38.excerpt 100%[===================>] 791.12K  --.-KB/s    in 0.06s   

2025-02-10 15:49:11 (12.4 MB/s) - ‘chr1.GRCh38.excerpt.fasta’ saved [810105/810105]



In [4]:
# Naive exact method - with the times the function did the comparisons
def naive_with_counts(whole_genome : str, reference : str):
    matching_offsets = []
    outer_loop_len = len(whole_genome) - len(reference) + 1
    times = 0
    alignments = 0
    for i in range(outer_loop_len):
        matched = True
        alignments += 1
        for j in range(len(reference)):
            times += 1
            if whole_genome[i + j] != reference[j]:
                matched = False
                break
        if matched:
            matching_offsets.append(i)
    return matching_offsets, alignments, times

In [5]:
from bm_preproc import BoyerMoore
def boyer_moore_with_counts(p, p_bm : BoyerMoore, t):
    """ Do Boyer-Moore matching. p=pattern, t=text,
        p_bm=BoyerMoore object for p """
    i = 0
    occurrences = []
    times = 0
    alignments = 0
    while i < len(t) - len(p) + 1:
        shift = 1
        mismatched = False
        alignments += 1
        for j in range(len(p)-1, -1, -1):
            times += 1
            if p[j] != t[i+j]:
                skip_bc = p_bm.bad_character_rule(j, t[i+j])
                skip_gs = p_bm.good_suffix_rule(j)
                shift = max(shift, skip_bc, skip_gs)
                mismatched = True
                break
        if not mismatched:
            occurrences.append(i)
            skip_gs = p_bm.match_skip()
            shift = max(shift, skip_gs)
        i += shift
    return occurrences, alignments, times

[40] 12 15
[0, 19] 5 18


In [6]:
p = 'word'
t = 'there would have been a time for such a word'
occurrences, num_alignments, num_character_comparisons = naive_with_counts(t, p)
print(occurrences, num_alignments, num_character_comparisons)

[40] 41 46


In [7]:
p = 'needle'
t = 'needle need noodle needle'
occurrences, num_alignments, num_character_comparisons = naive_with_counts(t, p)
print(occurrences, num_alignments, num_character_comparisons)

[0, 19] 20 35


In [8]:
# Question 1
p_human = "GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG"
with open("chr1.GRCh38.excerpt.fasta", "r") as f:
    global chrom_1_genome
    chrom_1_genome = "".join([line.rstrip() for line in f if line[0] != ">"])



In [9]:
occurrences, num_alignments, num_character_comparisons = naive_with_counts(chrom_1_genome, p_human)
print(occurrences, num_alignments, num_character_comparisons)

[56922] 799954 984143


In [10]:
# Question 2
p_bm = BoyerMoore(p_human, alphabet="ACGT")
occurrences, num_alignments, num_character_comparisons = boyer_moore_with_counts(p_human, p_bm, chrom_1_genome)
print(occurrences, num_alignments, num_character_comparisons)

[56922] 127974 165191


In [11]:
# Question 3
class Index(object):
    def __init__(self, t, k):
        ''' Create index from all substrings of size 'length' '''
        self.k = k  # k-mer length (k)
        self.index = []
        for i in range(len(t) - k + 1):  # for each k-mer
            self.index.append((t[i:i+k], i))  # add (k-mer, offset) pair
        self.index.sort()  # alphabetize by k-mer
    
    def query(self, p):
        ''' Return index hits for first k-mer of P '''
        kmer = p[:self.k]  # query with first k-mer
        i = bisect.bisect_left(self.index, (kmer, -1))  # binary search
        hits = []
        while i < len(self.index):  # collect matching index entries
            if self.index[i][0] != kmer:
                break
            hits.append(self.index[i][1])
            i += 1
        return hits

In [12]:
def approximate_match(pattern, text, max_num_mismatch):
    """
    divides the pattern into (max_num_mismatch + 1) partisans
    so at least one partisan will be intact
    """
    num_partisans = max_num_mismatch + 1
    length_per_partisan = round(len(pattern) / num_partisans)
    all_matches = set()
    for i in range(num_partisans):
        start = i * length_per_partisan
        end = min((i+1) * length_per_partisan, len(pattern))
        boyer_moore_obj = BoyerMoore(pattern[start:end], alphabet="ACGT")
        matches = boyer_moore(pattern[start:end], boyer_moore_obj, text)
        for m in matches:
            if m < start or m - start + len(pattern) > len(text):
                continue
            mismatches = 0
            for j in range(0, start):
                if pattern[j] != text[m - start + j]:
                    mismatches += 1
                    if mismatches > max_num_mismatch:
                        break
            for j_end in range(end, len(pattern)):
                if pattern[j_end] != text[m - start + j_end]:
                    mismatches += 1
                    if mismatches > max_num_mismatch:
                        break
            if mismatches <= 2:
                all_matches.add(m - start)
    return list(all_matches)

def boyer_moore(p, p_bm, t):
    """ Do Boyer-Moore matching """
    i = 0
    occurrences = []
    while i < len(t) - len(p) + 1:
        shift = 1
        mismatched = False
        for j in range(len(p)-1, -1, -1):
            if p[j] != t[i+j]:
                skip_bc = p_bm.bad_character_rule(j, t[i+j])
                skip_gs = p_bm.good_suffix_rule(j)
                shift = max(shift, skip_bc, skip_gs)
                mismatched = True
                break
        if not mismatched:
            occurrences.append(i)
            skip_gs = p_bm.match_skip()
            shift = max(shift, skip_gs)
        i += shift
    return occurrences

In [13]:
len(approximate_match("GGCGCGGTGGCTCACGCCTGTAAT", chrom_1_genome, 2))

19

In [14]:
# Question 4
def approximate_match_list(pattern, text, max_num_mismatch):
    """
    divides the pattern into (max_num_mismatch + 1) partisans
    so at least one partisan will be intact
    """
    num_partisans = max_num_mismatch + 1
    length_per_partisan = round(len(pattern) / num_partisans)
    all_matches = set()
    index_hits = 0
    for i in range(num_partisans):
        start = i * length_per_partisan
        end = min((i+1) * length_per_partisan, len(pattern))
        boyer_moore_obj = BoyerMoore(pattern[start:end], alphabet="ACGT")
        matches = boyer_moore(pattern[start:end], boyer_moore_obj, text)
        index_hits += len(matches) # index hits means how many results after the query but not verified yet!!
        for m in matches:
            if m < start or m - start + len(pattern) > len(text):
                continue
            mismatches = 0
            for j in range(0, start):
                if pattern[j] != text[m - start + j]:
                    mismatches += 1
                    if mismatches > max_num_mismatch:
                        break
            for j_end in range(end, len(pattern)):
                if pattern[j_end] != text[m - start + j_end]:
                    mismatches += 1
                    if mismatches > max_num_mismatch:
                        break
            if mismatches <= 2:
                all_matches.add(m - start)
    return all_matches, index_hits

In [15]:
occurences, hitsss = approximate_match_list("GGCGCGGTGGCTCACGCCTGTAAT", chrom_1_genome, 2)

In [16]:
hitsss

90

In [2]:
import bisect
   
class SubseqIndex(object):
    """ Holds a subsequence index for a text T """
    
    def __init__(self, t, k, ival):
        """ Create index from all subsequences consisting of k characters
            spaced ival positions apart.  E.g., SubseqIndex("ATAT", 2, 2)
            extracts ("AA", 0) and ("TT", 1). """
        self.k = k  # num characters per subsequence extracted
        self.ival = ival  # space between them; 1=adjacent, 2=every other, etc
        self.index = []
        self.span = 1 + ival * (k - 1)
        for i in range(len(t) - self.span + 1):  # for each subseq
            self.index.append((t[i:i+self.span:ival], i))  # add (subseq, offset)
        self.index.sort()  # alphabetize by subseq
    
    def query(self, p):
        """ Return index hits for first subseq of p """
        subseq = p[:self.span:self.ival]  # query with first subseq
        i = bisect.bisect_left(self.index, (subseq, -1))  # binary search
        hits = []
        while i < len(self.index):  # collect matching index entries
            if self.index[i][0] != subseq:
                break
            hits.append(self.index[i][1])
            i += 1
        return hits


In [3]:
# use pigeonhole
def approximate_subseq(pattern, text, subseq_obj: SubseqIndex, max_num_mismatches=2):
    occurrences = set()
    index_hits_total = 0
    if subseq_obj.k * subseq_obj.ival != len(pattern):
        raise RuntimeError("Subsequences cannot cover every character of the pattern")

    index_hits_per_subseq = subseq_obj.query(pattern) # get the 1st subseq
    
    for i in range(subseq_obj.ival):
        if i != 0:
            subseq = pattern[i:i+subseq_obj.span:subseq_obj.ival]
            raw_index = bisect.bisect_left(subseq_obj.index, (subseq, -1))  # binary search
            hits = []
            while raw_index < len(subseq_obj.index):  # collect matching index entries
                if subseq_obj.index[raw_index][0] != subseq:
                    break
                hits.append(subseq_obj.index[raw_index][1])
                raw_index += 1
            index_hits_per_subseq = hits
        index_hits_total += len(index_hits_per_subseq)
        for index_hit in index_hits_per_subseq:
            j = 0
            if index_hit - i < 0 or len(pattern) + index_hit > len(text):
                break
            mismatches = 0
            while j < len(pattern):
                if j - i % subseq_obj.ival == 0 and j - i <= subseq_obj.span:
                    j += 1
                    continue
                if pattern[j] != text[j + index_hit]:
                    mismatches += 1
                    if mismatches > max_num_mismatches:
                        break
                j += 1
            if mismatches <= max_num_mismatches:
                occurrences.add(index_hit - i)
            
    return occurrences, index_hits_total

In [56]:
t = chrom_1_genome
p = 'GGCGCGGTGGCTCACGCCTGTAAT'
sub_obj = SubseqIndex(t, 8, 3)
occurrences, hits = approximate_subseq(p, t, sub_obj)

In [57]:
occurrences

{56922,
 84641,
 147558,
 191452,
 262042,
 273669,
 364263,
 465647,
 635931,
 657496,
 681737,
 717706,
 747359}

In [58]:
hits

79

In [5]:
p = 'donner'
t = 'this way you have to go home to get a nice dinner'
sub_obj = SubseqIndex(t, 3, 2)
occurrences, hits = approximate_subseq(p, t, sub_obj)
occurrences

RuntimeError: Subsequences cannot cover every character of the pattern